# Week 17 - Day 7: Interview Review
## Options, Derivatives & Deep Hedging

**Objectives:**
- Master 10 essential options/derivatives interview questions
- Review key concepts from the week
- Practice coding implementations

---

## Interview Question 1: Explain Put-Call Parity

### Question
*"What is put-call parity and how would you exploit a violation?"*

### Answer
Put-call parity is a fundamental relationship between European call and put options:

$$C - P = S - Ke^{-rT}$$

Where:
- $C$ = Call price, $P$ = Put price
- $S$ = Spot price, $K$ = Strike price
- $r$ = Risk-free rate, $T$ = Time to expiry

**Arbitrage Strategy:**
- If $C - P > S - Ke^{-rT}$: Sell call, buy put, buy stock, borrow $Ke^{-rT}$
- If $C - P < S - Ke^{-rT}$: Buy call, sell put, short stock, lend $Ke^{-rT}$

In [ ]:
import numpy as np

def check_put_call_parity(C, P, S, K, r, T):
    """Check put-call parity and identify arbitrage."""
    left_side = C - P
    right_side = S - K * np.exp(-r * T)
    
    difference = left_side - right_side
    
    print(f"C - P = {left_side:.4f}")
    print(f"S - K*e^(-rT) = {right_side:.4f}")
    print(f"Difference = {difference:.4f}")
    
    if abs(difference) < 0.01:
        return "Parity holds - no arbitrage"
    elif difference > 0:
        return "Call overpriced: Sell call, buy put, buy stock"
    else:
        return "Put overpriced: Buy call, sell put, short stock"

# Example
print(check_put_call_parity(C=10.5, P=5.2, S=100, K=95, r=0.05, T=0.5))

## Interview Question 2: Black-Scholes Greeks

### Question
*"Explain Delta, Gamma, Theta, Vega, and Rho. Which is most important for a market maker?"*

### Answer

| Greek | Definition | Formula Intuition |
|-------|------------|-------------------|
| **Delta (Δ)** | Rate of change of option price w.r.t. spot | $\frac{\partial V}{\partial S}$ |
| **Gamma (Γ)** | Rate of change of delta w.r.t. spot | $\frac{\partial^2 V}{\partial S^2}$ |
| **Theta (Θ)** | Time decay per day | $\frac{\partial V}{\partial t}$ |
| **Vega (ν)** | Sensitivity to volatility | $\frac{\partial V}{\partial \sigma}$ |
| **Rho (ρ)** | Sensitivity to interest rates | $\frac{\partial V}{\partial r}$ |

**For Market Makers:** Gamma is most critical because:
1. High gamma = large hedging costs (frequent rebalancing)
2. Gamma exposure drives P&L from realized vs implied volatility
3. Short gamma positions can lead to significant losses in volatile markets

In [ ]:
from scipy.stats import norm

def black_scholes_greeks(S, K, T, r, sigma, option_type='call'):
    """Calculate all Black-Scholes Greeks."""
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    # Common terms
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    n_d1 = norm.pdf(d1)
    
    if option_type == 'call':
        delta = N_d1
        theta = (-S*n_d1*sigma/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*N_d2) / 365
        rho = K*T*np.exp(-r*T)*N_d2 / 100
    else:
        delta = N_d1 - 1
        theta = (-S*n_d1*sigma/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*norm.cdf(-d2)) / 365
        rho = -K*T*np.exp(-r*T)*norm.cdf(-d2) / 100
    
    gamma = n_d1 / (S*sigma*np.sqrt(T))
    vega = S*n_d1*np.sqrt(T) / 100
    
    return {'Delta': delta, 'Gamma': gamma, 'Theta': theta, 'Vega': vega, 'Rho': rho}

# Example: ATM call
greeks = black_scholes_greeks(S=100, K=100, T=0.25, r=0.05, sigma=0.2)
for greek, value in greeks.items():
    print(f"{greek}: {value:.4f}")

## Interview Question 3: Implied vs Realized Volatility

### Question
*"How do you trade the spread between implied and realized volatility?"*

### Answer

**Volatility Trading Strategies:**

1. **IV > RV Expected (Volatility is rich):**
   - Sell options (straddles/strangles)
   - Delta-hedge to isolate volatility exposure
   - Profit from theta decay if realized vol stays low

2. **IV < RV Expected (Volatility is cheap):**
   - Buy options (straddles/strangles)
   - Gamma scalping: profit from large moves

**Key P&L Relationship:**
$$P\&L \approx \frac{1}{2}\Gamma S^2 (\sigma_{realized}^2 - \sigma_{implied}^2) \cdot T$$

In [ ]:
def calculate_realized_vol(returns, window=21, annualize=252):
    """Calculate realized volatility from returns."""
    return returns.rolling(window).std() * np.sqrt(annualize)

def vol_trading_signal(implied_vol, realized_vol, threshold=0.02):
    """Generate volatility trading signal."""
    spread = implied_vol - realized_vol
    
    if spread > threshold:
        return "SELL VOL: IV rich, sell straddle"
    elif spread < -threshold:
        return "BUY VOL: IV cheap, buy straddle"
    else:
        return "NEUTRAL: No clear edge"

# Example
print(vol_trading_signal(implied_vol=0.25, realized_vol=0.18))
print(vol_trading_signal(implied_vol=0.15, realized_vol=0.22))

## Interview Question 4: Volatility Smile/Skew

### Question
*"Why does the volatility smile exist? What does skew tell you about market sentiment?"*

### Answer

**Why the Smile Exists:**
1. **Fat tails:** Real returns have fatter tails than Black-Scholes assumes
2. **Jump risk:** Markets can gap, especially downward
3. **Leverage effect:** Volatility increases when prices fall
4. **Supply/demand:** Institutions buy OTM puts for protection

**Skew Interpretation:**
- **Steep negative skew:** Market fears downside (equity markets)
- **Positive skew:** Market fears upside (commodities, short squeezes)
- **Flat smile:** Market expects symmetric moves

**Skew Metrics:**
$$Skew = IV_{25\Delta Put} - IV_{25\Delta Call}$$

In [ ]:
def analyze_vol_surface(strikes, ivs, atm_strike):
    """Analyze volatility surface characteristics."""
    # Find ATM IV
    atm_idx = np.argmin(np.abs(np.array(strikes) - atm_strike))
    atm_iv = ivs[atm_idx]
    
    # Calculate skew (25 delta approximation)
    otm_put_iv = ivs[0]   # Low strike
    otm_call_iv = ivs[-1]  # High strike
    
    skew = otm_put_iv - otm_call_iv
    smile = (otm_put_iv + otm_call_iv) / 2 - atm_iv
    
    print(f"ATM IV: {atm_iv:.2%}")
    print(f"Skew (Put - Call): {skew:.2%}")
    print(f"Smile (Wings - ATM): {smile:.2%}")
    
    if skew > 0.02:
        print("Market sentiment: Bearish (demand for downside protection)")
    elif skew < -0.02:
        print("Market sentiment: Bullish (demand for upside)")

# Example: Typical equity skew
strikes = [90, 95, 100, 105, 110]
ivs = [0.28, 0.23, 0.20, 0.19, 0.18]
analyze_vol_surface(strikes, ivs, atm_strike=100)

## Interview Question 5: Delta Hedging

### Question
*"Explain delta hedging. What are the costs and when does it fail?"*

### Answer

**Delta Hedging Process:**
1. Calculate option delta
2. Take opposite position in underlying: $Hedge = -\Delta \times Contracts \times Multiplier$
3. Rebalance as delta changes

**Costs:**
- Transaction costs from frequent rebalancing
- Bid-ask spread
- Gamma exposure (discrete hedging misses moves between rebalances)

**When It Fails:**
- **Jumps/gaps:** Overnight gaps bypass delta hedge
- **High gamma:** Near expiry, ATM options have unstable deltas
- **Liquidity crises:** Can't execute hedges at fair prices
- **Model risk:** Wrong volatility estimate → wrong delta

In [ ]:
def simulate_delta_hedge(S0, K, T, r, sigma, n_steps, n_paths=1000):
    """Simulate delta hedging P&L."""
    dt = T / n_steps
    
    # Initial option price and delta
    d1 = (np.log(S0/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    option_price = S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d1 - sigma*np.sqrt(T))
    
    hedge_pnls = []
    
    for _ in range(n_paths):
        S = S0
        cash = option_price  # Received premium
        stock_held = -norm.cdf(d1)  # Short delta shares
        cash -= stock_held * S0  # Buy shares for hedge
        
        # Simulate path
        for i in range(n_steps):
            t_remaining = T - i*dt
            if t_remaining <= 0:
                break
                
            # Price move
            dW = np.random.normal(0, np.sqrt(dt))
            S_new = S * np.exp((r - 0.5*sigma**2)*dt + sigma*dW)
            
            # New delta
            d1_new = (np.log(S_new/K) + (r + 0.5*sigma**2)*t_remaining) / (sigma*np.sqrt(t_remaining))
            new_delta = -norm.cdf(d1_new)
            
            # Rebalance
            shares_to_trade = new_delta - stock_held
            cash -= shares_to_trade * S_new
            stock_held = new_delta
            S = S_new
        
        # Final P&L
        payoff = max(S - K, 0)
        final_pnl = cash + stock_held * S - payoff
        hedge_pnls.append(final_pnl)
    
    print(f"Mean Hedge P&L: ${np.mean(hedge_pnls):.4f}")
    print(f"Std Hedge P&L: ${np.std(hedge_pnls):.4f}")
    return hedge_pnls

# Simulate with daily rebalancing
np.random.seed(42)
pnls = simulate_delta_hedge(S0=100, K=100, T=0.25, r=0.05, sigma=0.2, n_steps=63)

## Interview Question 6: Option Pricing Models

### Question
*"Compare Black-Scholes, Binomial, and Monte Carlo. When would you use each?"*

### Answer

| Model | Best For | Limitations |
|-------|----------|-------------|
| **Black-Scholes** | European options, quick pricing | No early exercise, constant vol |
| **Binomial** | American options, discrete dividends | Slow for many steps, path-independent only |
| **Monte Carlo** | Path-dependent, exotic options | Slow for Americans, variance issues |

**Use Cases:**
- **Black-Scholes:** Vanilla European calls/puts, Greeks calculation
- **Binomial:** American options, convertible bonds
- **Monte Carlo:** Asian options, barrier options, basket options

In [ ]:
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    """Black-Scholes European option price."""
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type == 'call':
        return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

def binomial_price(S, K, T, r, sigma, n_steps, option_type='call', american=False):
    """Binomial tree option pricing."""
    dt = T / n_steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r*dt) - d) / (u - d)
    
    # Terminal prices
    prices = S * u**np.arange(n_steps, -1, -1) * d**np.arange(0, n_steps+1)
    
    # Terminal payoffs
    if option_type == 'call':
        values = np.maximum(prices - K, 0)
    else:
        values = np.maximum(K - prices, 0)
    
    # Backward induction
    for i in range(n_steps-1, -1, -1):
        prices = S * u**np.arange(i, -1, -1) * d**np.arange(0, i+1)
        values = np.exp(-r*dt) * (p*values[:-1] + (1-p)*values[1:])
        
        if american:
            if option_type == 'call':
                values = np.maximum(values, prices - K)
            else:
                values = np.maximum(values, K - prices)
    
    return values[0]

# Compare methods
S, K, T, r, sigma = 100, 100, 0.25, 0.05, 0.2

print(f"Black-Scholes Call: ${black_scholes_price(S, K, T, r, sigma):.4f}")
print(f"Binomial Call (100 steps): ${binomial_price(S, K, T, r, sigma, 100):.4f}")
print(f"American Put (Binomial): ${binomial_price(S, K, T, r, sigma, 100, 'put', True):.4f}")
print(f"European Put (Binomial): ${binomial_price(S, K, T, r, sigma, 100, 'put', False):.4f}")

## Interview Question 7: Deep Hedging Concept

### Question
*"What is deep hedging and how does it differ from traditional delta hedging?"*

### Answer

**Traditional Delta Hedging:**
- Model-based (Black-Scholes assumptions)
- Ignores transaction costs
- Continuous rebalancing (theoretical)
- Greeks computed analytically

**Deep Hedging:**
- Data-driven (neural network learns hedge ratios)
- Incorporates transaction costs in objective
- Learns optimal discrete rebalancing
- No model assumptions required

**Deep Hedging Objective:**
$$\min_{\theta} \mathbb{E}\left[\rho\left(P\&L - \sum_{t} \delta_t^{\theta}(S_{t+1} - S_t) - TC\right)\right]$$

Where $\rho$ is a risk measure (e.g., CVaR) and $\delta_t^{\theta}$ is the neural network hedge ratio.

In [ ]:
# Simplified Deep Hedging concept (pseudo-implementation)

class SimpleDeepHedger:
    """Conceptual deep hedging model."""
    
    def __init__(self, transaction_cost=0.001):
        self.tc = transaction_cost
        # In practice: neural network weights
        self.weights = np.random.randn(3) * 0.1
    
    def compute_hedge(self, S, K, T, sigma):
        """Compute hedge ratio from features."""
        # Features: moneyness, time, vol
        features = np.array([S/K - 1, T, sigma])
        # Simple linear model (in practice: deep network)
        hedge = np.tanh(np.dot(self.weights, features))
        return hedge
    
    def hedge_pnl(self, S_path, K, T, sigma):
        """Compute P&L for a price path."""
        n_steps = len(S_path) - 1
        dt = T / n_steps
        
        pnl = 0
        prev_hedge = 0
        
        for i in range(n_steps):
            t_remaining = T - i*dt
            hedge = self.compute_hedge(S_path[i], K, t_remaining, sigma)
            
            # P&L from hedge
            pnl += hedge * (S_path[i+1] - S_path[i])
            
            # Transaction cost
            pnl -= self.tc * abs(hedge - prev_hedge) * S_path[i]
            prev_hedge = hedge
        
        # Option payoff (short call)
        payoff = max(S_path[-1] - K, 0)
        
        return pnl - payoff

print("Deep Hedging Key Idea:")
print("- Learn hedge ratios directly from data")
print("- Optimize for risk-adjusted P&L including costs")
print("- No reliance on Black-Scholes assumptions")

## Interview Question 8: Exotic Options

### Question
*"Describe barrier options and Asian options. How do you price them?"*

### Answer

**Barrier Options:**
- Activated or deactivated when price crosses a barrier
- Types: Knock-in, Knock-out, Up, Down
- Pricing: Analytical (simple barriers) or Monte Carlo
- Hedging challenge: Gamma explodes near barrier

**Asian Options:**
- Payoff based on average price over period
- Types: Arithmetic average (no closed form), Geometric average (has formula)
- Lower premium than vanilla (averaging reduces volatility)
- Pricing: Monte Carlo for arithmetic, analytical for geometric

In [ ]:
def monte_carlo_asian(S0, K, T, r, sigma, n_paths=10000, n_steps=252):
    """Price arithmetic Asian call option."""
    dt = T / n_steps
    
    payoffs = []
    for _ in range(n_paths):
        # Simulate price path
        Z = np.random.normal(0, 1, n_steps)
        S = S0 * np.exp(np.cumsum((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z))
        
        # Average price
        avg_price = np.mean(S)
        payoff = max(avg_price - K, 0)
        payoffs.append(payoff)
    
    price = np.exp(-r*T) * np.mean(payoffs)
    std_err = np.exp(-r*T) * np.std(payoffs) / np.sqrt(n_paths)
    
    return price, std_err

def monte_carlo_barrier(S0, K, H, T, r, sigma, barrier_type='down-out', n_paths=10000, n_steps=252):
    """Price barrier option (down-and-out call)."""
    dt = T / n_steps
    
    payoffs = []
    for _ in range(n_paths):
        S = S0
        knocked_out = False
        
        for _ in range(n_steps):
            Z = np.random.normal()
            S = S * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z)
            
            if barrier_type == 'down-out' and S <= H:
                knocked_out = True
                break
        
        if knocked_out:
            payoffs.append(0)
        else:
            payoffs.append(max(S - K, 0))
    
    return np.exp(-r*T) * np.mean(payoffs)

np.random.seed(42)

# Price exotic options
asian_price, asian_se = monte_carlo_asian(S0=100, K=100, T=1, r=0.05, sigma=0.2)
print(f"Asian Call Price: ${asian_price:.4f} (±{asian_se:.4f})")

barrier_price = monte_carlo_barrier(S0=100, K=100, H=80, T=1, r=0.05, sigma=0.2)
vanilla_price = black_scholes_price(100, 100, 1, 0.05, 0.2)
print(f"Down-Out Call (H=80): ${barrier_price:.4f}")
print(f"Vanilla Call: ${vanilla_price:.4f}")

## Interview Question 9: Risk Management

### Question
*"How do you calculate VaR for an options portfolio? What are the limitations?"*

### Answer

**VaR Calculation Methods:**

1. **Delta-Normal VaR:**
   - $VaR = \Delta \times S \times \sigma \times z_{\alpha} \times \sqrt{T}$
   - Fast but ignores gamma, skew

2. **Delta-Gamma VaR:**
   - Includes second-order term: $\frac{1}{2}\Gamma S^2 \sigma^2 T$
   - Better for large moves

3. **Full Revaluation (Monte Carlo):**
   - Simulate price scenarios
   - Reprice portfolio for each scenario
   - Most accurate but slowest

**Limitations:**
- VaR doesn't capture tail risk (use CVaR/ES instead)
- Assumes normal distributions
- Backward-looking
- Ignores liquidity risk

In [ ]:
def options_portfolio_var(positions, S, K_list, T, r, sigma, confidence=0.95, method='delta-gamma'):
    """
    Calculate VaR for options portfolio.
    positions: list of (quantity, strike, option_type)
    """
    z = norm.ppf(confidence)
    daily_vol = sigma / np.sqrt(252)
    
    total_delta = 0
    total_gamma = 0
    total_vega = 0
    
    for qty, K, opt_type in positions:
        greeks = black_scholes_greeks(S, K, T, r, sigma, opt_type)
        total_delta += qty * greeks['Delta'] * 100  # 100 shares per contract
        total_gamma += qty * greeks['Gamma'] * 100
        total_vega += qty * greeks['Vega'] * 100
    
    # Delta-only VaR
    delta_var = abs(total_delta * S * daily_vol * z)
    
    # Delta-Gamma VaR (Cornish-Fisher expansion)
    if method == 'delta-gamma':
        gamma_adjustment = 0.5 * total_gamma * (S * daily_vol * z)**2
        dg_var = delta_var + abs(gamma_adjustment)
        return dg_var
    
    return delta_var

# Example portfolio: Long 10 calls, short 5 puts
portfolio = [
    (10, 100, 'call'),   # Long 10 ATM calls
    (-5, 95, 'put'),     # Short 5 OTM puts
]

var_95 = options_portfolio_var(portfolio, S=100, K_list=[100, 95], T=0.25, r=0.05, sigma=0.2)
print(f"1-Day 95% Delta-Gamma VaR: ${var_95:.2f}")

## Interview Question 10: Market Making

### Question
*"How does an options market maker make money and manage risk?"*

### Answer

**Revenue Sources:**
1. **Bid-ask spread:** Buy at bid, sell at ask
2. **Volatility edge:** Buy when IV < expected RV, sell when IV > expected RV
3. **Flow information:** Understanding customer positioning

**Risk Management:**
1. **Delta hedge:** Neutralize directional exposure
2. **Gamma limits:** Control exposure to large moves
3. **Vega limits:** Manage volatility exposure
4. **Inventory management:** Don't accumulate large positions

**Key Metrics:**
- Net delta, gamma, vega exposure
- P&L attribution (spread vs volatility)
- Fill rates and adverse selection

In [ ]:
class SimpleMarketMaker:
    """Simplified options market maker simulation."""
    
    def __init__(self, spread_pct=0.02, gamma_limit=100, delta_limit=500):
        self.spread_pct = spread_pct
        self.gamma_limit = gamma_limit
        self.delta_limit = delta_limit
        self.inventory = []  # List of positions
        self.pnl = 0
    
    def quote(self, S, K, T, r, sigma):
        """Generate bid/ask quotes."""
        fair_value = black_scholes_price(S, K, T, r, sigma)
        half_spread = fair_value * self.spread_pct / 2
        
        bid = fair_value - half_spread
        ask = fair_value + half_spread
        
        return bid, ask, fair_value
    
    def check_risk_limits(self, S, K, T, r, sigma, side):
        """Check if trade would breach risk limits."""
        greeks = black_scholes_greeks(S, K, T, r, sigma)
        
        # Calculate current portfolio greeks
        current_delta = sum(p['delta'] for p in self.inventory)
        current_gamma = sum(p['gamma'] for p in self.inventory)
        
        # Projected greeks after trade
        direction = 1 if side == 'buy' else -1
        new_delta = current_delta + direction * greeks['Delta'] * 100
        new_gamma = current_gamma + direction * greeks['Gamma'] * 100
        
        if abs(new_delta) > self.delta_limit:
            return False, "Delta limit breached"
        if abs(new_gamma) > self.gamma_limit:
            return False, "Gamma limit breached"
        
        return True, "OK"
    
    def get_portfolio_summary(self):
        """Get portfolio risk summary."""
        total_delta = sum(p['delta'] for p in self.inventory)
        total_gamma = sum(p['gamma'] for p in self.inventory)
        total_vega = sum(p['vega'] for p in self.inventory)
        
        return {
            'positions': len(self.inventory),
            'delta': total_delta,
            'gamma': total_gamma,
            'vega': total_vega,
            'pnl': self.pnl
        }

# Demonstrate market maker
mm = SimpleMarketMaker()
bid, ask, fair = mm.quote(S=100, K=100, T=0.25, r=0.05, sigma=0.2)
print(f"Fair Value: ${fair:.4f}")
print(f"Bid: ${bid:.4f}")
print(f"Ask: ${ask:.4f}")
print(f"Spread: ${ask-bid:.4f} ({(ask-bid)/fair*100:.2f}%)")

---

## Week 17 Summary: Options, Derivatives & Deep Hedging

### Key Concepts Covered

| Day | Topic | Key Takeaways |
|-----|-------|---------------|
| 1 | Options Basics | Put-call parity, payoff diagrams, moneyness |
| 2 | Black-Scholes | Pricing formula, assumptions, Greeks |
| 3 | Volatility | Implied vs realized, smile/skew, term structure |
| 4 | Delta Hedging | Traditional hedging, rebalancing, P&L |
| 5 | Deep Hedging | Neural networks for hedging, transaction costs |
| 6 | Exotic Options | Barriers, Asians, Monte Carlo pricing |
| 7 | Interview Review | 10 key questions, practical implementations |

### Essential Formulas

1. **Black-Scholes Call:** $C = SN(d_1) - Ke^{-rT}N(d_2)$

2. **Put-Call Parity:** $C - P = S - Ke^{-rT}$

3. **Delta:** $\Delta_{call} = N(d_1)$, $\Delta_{put} = N(d_1) - 1$

4. **Gamma P&L:** $P\&L \approx \frac{1}{2}\Gamma (\Delta S)^2$

### Interview Tips

1. **Know your Greeks** - Be able to explain intuitively and mathematically
2. **Understand volatility** - Smile, skew, term structure, realized vs implied
3. **Risk management** - VaR limitations, position limits, hedging costs
4. **Deep hedging** - Know the concept even if implementation is complex
5. **Market making** - Understand how MMs profit and manage risk

In [ ]:
# Quick Reference: All Key Functions

print("="*50)
print("WEEK 17 QUICK REFERENCE")
print("="*50)

# Example calculations
S, K, T, r, sigma = 100, 100, 0.25, 0.05, 0.20

print("\n1. BLACK-SCHOLES PRICING")
call_price = black_scholes_price(S, K, T, r, sigma, 'call')
put_price = black_scholes_price(S, K, T, r, sigma, 'put')
print(f"   Call: ${call_price:.4f}, Put: ${put_price:.4f}")

print("\n2. GREEKS (ATM Call)")
greeks = black_scholes_greeks(S, K, T, r, sigma)
for g, v in greeks.items():
    print(f"   {g}: {v:.4f}")

print("\n3. PUT-CALL PARITY CHECK")
parity = check_put_call_parity(call_price, put_price, S, K, r, T)

print("\n4. VOLATILITY TRADING")
print(f"   {vol_trading_signal(0.25, 0.18)}")

print("\n" + "="*50)
print("Week 17 Complete! Ready for Week 18: Portfolio Optimization")
print("="*50)